<a href="https://colab.research.google.com/github/sacculifer/NLPower-capstone-project/blob/Alex/NER_Alex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install spacy
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.6 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 34.4 MB/s 
     |████████████████████████████████| 6.6 MB 40.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [191]:
import pandas as pd

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from google.colab import files

In [3]:
df = pd.read_csv("train.csv")

In [176]:
tweets = df['text']

In [177]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy='average')
#nlp = pipeline("ner", aggregation_strategy='average')

In [179]:
ner_results = nlp(list(tweets))
ner_results = pd.DataFrame(ner_results)

In [180]:
ner_results['locations'] = ""
for i in range(len(ner_results[0])):
  ner_results['locations'][i] = []
  for j in range(11):
    if ner_results[j][i] is None:
      continue
    if ner_results[j][i]['entity_group'] == 'LOC' and ner_results[j][i]['score'] > 0.8:
      ner_results['locations'][i].append(ner_results[j][i]['word'])
    else:
      continue

In [ ]:
ner_results ["empty"] = ""

In [188]:
ner_results

,0,1,2,3,4,5,6,7,8,9,10,text,locations,location_1,lat1,long1,country,state,city
0,None,None,None,None,None,None,None,None,None,None,None,Our Deeds are the Reason of this #earthquake M...,[],None,,,,,
1,"{'entity_group': 'LOC', 'score': 0.99139684, '...","{'entity_group': 'LOC', 'score': 0.9997851, 'w...",None,None,None,None,None,None,None,None,None,Forest fire near La Ronge Sask. Canada,"[La Ronge Sask, Canada]",La Ronge Sask,55.100528,-105.290048,Canada,Saskatchewan,
2,None,None,None,None,None,None,None,None,None,None,None,All residents asked to 'shelter in place' are ...,[],None,,,,,
3,"{'entity_group': 'LOC', 'score': 0.99977106, '...",None,None,None,None,None,None,None,None,None,None,"13,000 people receive #wildfires evacuation or...",[California],California,36.701463,-118.755997,United States,California,
4,"{'entity_group': 'LOC', 'score': 0.3436687, 'w...","{'entity_group': 'LOC', 'score': 0.9913408, 'w...",None,None,None,None,None,None,None,None,None,Just got sent this photo from Ruby #Alaska as ...,[Alaska],Alaska,64.445961,-149.680909,United States,Alaska,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,None,None,None,None,None,None,None,None,None,None,None,Two giant cranes holding a bridge collapse int...,[],None,,,,,
7609,"{'entity_group': 'LOC', 'score': 0.999671, 'wo...","{'entity_group': 'MISC', 'score': 0.7334061, '...",None,None,None,None,None,None,None,None,None,@aria_ahrary @TheTawniest The out of control w...,[California],California,,,,,
7610,"{'entity_group': 'ORG', 'score': 0.3611209, 'w...","{'entity_group': 'LOC', 'score': 0.86989534, '...",None,None,None,None,None,None,None,None,None,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,[Hawaii],Hawaii,,,,,
7611,"{'entity_group': 'LOC', 'score': 0.99159527, '...",None,None,None,None,None,None,None,None,None,None,Police investigating after an e-bike collided ...,[Little Portugal],Little Portugal,,,,,


In [182]:
ner_results['location_1'] = [next(iter(l), None) for l in ner_results['locations']]

In [184]:
ner_results['lat1'] = ""
ner_results['long1'] = ""
for i in range(len(ner_results)):
  if ner_results['location_1'][i] is None:
    continue
  else:
    #print(geolocator.geocode(ner_results['location_1'][i]).latitude)
    ner_results['lat1'][i] = geolocator.geocode(ner_results['location_1'][i]).latitude
    ner_results['long1'][i] = geolocator.geocode(ner_results['location_1'][i]).longitude

AttributeError: ignored

In [187]:
ner_results['country'] = ""
ner_results['state'] = ""
ner_results['city'] = ""

for i in range(len(ner_results)):
  if ner_results['location_1'][i] is None:
    continue
  else:
    location = geolocator.reverse(str(ner_results['lat1'][i]) + " , " + str(ner_results['long1'][i]))
    address = location.raw['address']
    ner_results['city'][i] = address.get('city', '')
    ner_results['state'][i] = address.get('state', '')
    ner_results['country'][i] = address.get('country', '')

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/base.py:256: UserWarning: Unable to parse the string as Point: "Failed to create Point instance from string: unknown format.". Using the value as-is for the query. In geopy 2.0 this will become an exception.
  'exception.' % str(e), UserWarning


GeocoderQueryError: ignored

In [192]:
ner_results.to_csv('output.csv')
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Latitude & Longitude input
coordinates = "17.3850 , 78.4867"

location = geolocator.reverse(coordinates)

address = location.raw['address']

# Traverse the data
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')

In [143]:
# Import the required library
from geopy.geocoders import Nominatim

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

location = geolocator.geocode()

print("The latitude of the location is: ", location.latitude)
print("The longitude of the location is: ", location.longitude)

The latitude of the location is:  44.933143
The longitude of the location is:  7.540121


In [129]:
from geopy.geocoders import Nominatim

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

# Latitude & Longitude input
coordinates = "17.3850 , 78.4867"

location = geolocator.reverse(coordinates)

address = location.raw['address']

# Traverse the data
city = address.get('city', '')
state = address.get('state', '')
country = address.get('country', '')

# Create a Label widget
print("Given Latitude and Longitude: " + coordinates)

print("The city is: " + city)

print("The state is: " + state)

print("The country is: " + country)

Given Latitude and Longitude: 17.3850 , 78.4867
The city is: Hyderabad
The state is: Telangana
The country is: India


In [114]:
import spacy
nlp = spacy.load("en_core_web_sm", disable = ["tagger","parser"])

#df = pd.DataFrame({"Text":["this is a text about Germany","this is another about Trump"]})

texts = df["text"].to_list()
ents = []
mylist = []
x = 0
for doc in nlp.pipe(texts):
    for ent in doc.ents:
        if ent.label_ == "GPE":
            ents.append(ent)
    mylist.append(ents)
    ents = []

/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
